In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import table

In [2]:
dataset = pd.read_csv('Data/zurich.csv')

# Count duplicates
dataset.groupby(dataset.columns.tolist(),as_index=False).size() # No duplicates found

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,size


In [3]:
# Count missing values
dataset_nans = dataset.copy()
for col in dataset_nans.columns:
    dataset_nans[col].replace('', np.nan, inplace=True)
    dataset_nans[col].replace(0, np.nan, inplace=True)
    dataset_nans[col].replace(float(0), np.nan, inplace=True)
dataset_nans.isna().sum()

id                                   0
name                                 1
host_id                              0
host_name                            0
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                1
minimum_nights                       0
number_of_reviews                  445
last_review                        445
reviews_per_month                  445
calculated_host_listings_count       0
availability_365                   474
number_of_reviews_ltm              844
license                           2246
dtype: int64

In [4]:
dataset = (dataset.set_index('id'))
dataset.drop("host_id", axis=1).describe()

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
count,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,1801.000000,2246.000000,2246.000000,2246.000000,0.0
mean,47.379313,8.534556,186.762244,12.136687,29.375779,1.229384,9.487979,150.410062,8.377115,NaN
std,0.023104,0.028604,287.972166,65.123043,65.484594,1.650677,19.182550,137.109183,16.995867,NaN
min,47.304080,8.442150,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000,NaN
25%,47.365150,8.518722,83.000000,1.000000,1.000000,0.200000,1.000000,7.000000,0.000000,NaN
50%,47.376820,8.532615,120.000000,3.000000,6.000000,0.570000,1.000000,113.000000,2.000000,NaN
75%,47.392917,8.550437,199.000000,7.000000,24.000000,1.510000,8.000000,294.000000,8.000000,NaN
max,47.452330,8.644700,9840.000000,1125.000000,903.000000,12.630000,91.000000,365.000000,224.000000,NaN


In [12]:
# import plotly.figure_factory as ff

# fig =  ff.create_table(dataset.drop("host_id", axis=1).describe().round(decimals=2).reset_index())
# fig.update_layout(
#     autosize=True,
#     #width=2000,
#     #height=200,
# )
# fig.write_image("table_plotly.png", scale=2)
# fig.show()

dataset.drop("host_id", axis=1).describe().round(decimals=2).reset_index().to_latex()

'\\begin{tabular}{llrrrrrrrrrr}\n\\toprule\n & index & latitude & longitude & price & minimum_nights & number_of_reviews & reviews_per_month & calculated_host_listings_count & availability_365 & number_of_reviews_ltm & license \\\\\n\\midrule\n0 & count & 2246.000000 & 2246.000000 & 2246.000000 & 2246.000000 & 2246.000000 & 1801.000000 & 2246.000000 & 2246.000000 & 2246.000000 & 0.000000 \\\\\n1 & mean & 47.380000 & 8.530000 & 186.760000 & 12.140000 & 29.380000 & 1.230000 & 9.490000 & 150.410000 & 8.380000 & NaN \\\\\n2 & std & 0.020000 & 0.030000 & 287.970000 & 65.120000 & 65.480000 & 1.650000 & 19.180000 & 137.110000 & 17.000000 & NaN \\\\\n3 & min & 47.300000 & 8.440000 & 0.000000 & 1.000000 & 0.000000 & 0.010000 & 1.000000 & 0.000000 & 0.000000 & NaN \\\\\n4 & 25% & 47.370000 & 8.520000 & 83.000000 & 1.000000 & 1.000000 & 0.200000 & 1.000000 & 7.000000 & 0.000000 & NaN \\\\\n5 & 50% & 47.380000 & 8.530000 & 120.000000 & 3.000000 & 6.000000 & 0.570000 & 1.000000 & 113.000000 & 2.000